In [ ]:
import pandas as pd
import pickle
from sklearn.preprocessing import MinMaxScaler

In [ ]:
train_data = pd.read_pickle('../../data/train_long_data.pkl')
val_long_data = pd.read_pickle('../../data/val_long_data.pkl')
test_long_data = pd.read_pickle('../../data/test_long_data.pkl')
outcomes = pd.read_pickle('../../data/SSI_outcomes.pkl')
with open('../../data/feature_selection_50_columns.pkl', 'rb') as f:
    keep_columns = pickle.load(f)

# Train Set

### Aggregate data for each patient and feature
Categorical features (like meds, micro results, encounters, dx and px codes) do a count; impute with 0

Numerical features (labs and vitals, terminology = LOINC) get mean, median, min, max; impute with median

In [ ]:
numerical = train_data.loc[train_data['TERMINOLOGY'] == 'LOINC'].copy()

median = numerical[['PT_KEY','FEATURE','VALUE']].groupby(['PT_KEY','FEATURE']).median().reset_index()
median['FEATURE'] = median['FEATURE'] + '_MEDIAN'
median = median.pivot_table(values='VALUE', index='PT_KEY', columns='FEATURE').reset_index()

mean = numerical[['PT_KEY','FEATURE','VALUE']].groupby(['PT_KEY','FEATURE']).mean().reset_index()
mean['FEATURE'] = mean['FEATURE'] + '_MEAN'
mean = mean.pivot_table(values='VALUE', index='PT_KEY', columns='FEATURE').reset_index()

minimum = numerical[['PT_KEY','FEATURE','VALUE']].groupby(['PT_KEY','FEATURE']).min().reset_index()
minimum['FEATURE'] = minimum['FEATURE'] + '_MIN'
minimum = minimum.pivot_table(values='VALUE', index='PT_KEY', columns='FEATURE').reset_index()

maximum = numerical[['PT_KEY','FEATURE','VALUE']].groupby(['PT_KEY','FEATURE']).max().reset_index()
maximum['FEATURE'] = maximum['FEATURE'] + '_MAX'
maximum = maximum.pivot_table(values='VALUE', index='PT_KEY', columns='FEATURE').reset_index()

train_medians = pd.concat([median.median(numeric_only=True), 
                           mean.median(numeric_only=True), 
                           minimum.median(numeric_only=True), 
                           maximum.median(numeric_only=True)])

In [ ]:
categorical = train_data.loc[train_data['TERMINOLOGY'] != 'LOINC'].copy()

count = categorical[['PT_KEY','FEATURE','VALUE']].groupby(['PT_KEY','FEATURE']).sum().reset_index()
count = count.pivot_table(values='VALUE', index='PT_KEY', columns='FEATURE').reset_index()

In [ ]:
agg_train = train_data[['PT_KEY']].drop_duplicates().merge(count, how='left', on='PT_KEY')
agg_train = agg_train.merge(median, how='left', on='PT_KEY')
agg_train = agg_train.merge(mean, how='left', on='PT_KEY')
agg_train = agg_train.merge(minimum, how='left', on='PT_KEY')
agg_train = agg_train.merge(maximum, how='left', on='PT_KEY')

agg_train = agg_train.fillna(train_medians.to_dict())
agg_train = agg_train.fillna(0)

agg_train = agg_train.merge(outcomes, how='inner', on='PT_KEY')

### Keep selected columns

In [ ]:
X_train=agg_train[keep_columns].copy()
y_train=agg_train['SSI']

### Scale data

In [ ]:
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train_scaled = pd.DataFrame(scaler.transform(X_train), 
                              columns = X_train.columns)

### Save data

In [ ]:
# X_train_scaled.to_pickle('../../data/train_X_agg.pkl')
# y_train.to_pickle('../../data/train_y_agg.pkl')

# Validation Set
### Aggregate data for each patient and feature
Categorical features (like meds, micro results, encounters, dx and px codes) do a count; impute with 0

Numerical features (labs and vitals, terminology = LOINC) get mean, median, min, max; impute with median

In [ ]:
numerical = val_long_data.loc[val_long_data['TERMINOLOGY'] == 'LOINC'].copy()

median = numerical[['PT_KEY','FEATURE','VALUE']].groupby(['PT_KEY','FEATURE']).median().reset_index()
median['FEATURE'] = median['FEATURE'] + '_MEDIAN'
median = median.pivot_table(values='VALUE', index='PT_KEY', columns='FEATURE').reset_index()

mean = numerical[['PT_KEY','FEATURE','VALUE']].groupby(['PT_KEY','FEATURE']).mean().reset_index()
mean['FEATURE'] = mean['FEATURE'] + '_MEAN'
mean = mean.pivot_table(values='VALUE', index='PT_KEY', columns='FEATURE').reset_index()

minimum = numerical[['PT_KEY','FEATURE','VALUE']].groupby(['PT_KEY','FEATURE']).min().reset_index()
minimum['FEATURE'] = minimum['FEATURE'] + '_MIN'
minimum = minimum.pivot_table(values='VALUE', index='PT_KEY', columns='FEATURE').reset_index()

maximum = numerical[['PT_KEY','FEATURE','VALUE']].groupby(['PT_KEY','FEATURE']).max().reset_index()
maximum['FEATURE'] = maximum['FEATURE'] + '_MAX'
maximum = maximum.pivot_table(values='VALUE', index='PT_KEY', columns='FEATURE').reset_index()

In [ ]:
categorical = val_long_data.loc[val_long_data['TERMINOLOGY'] != 'LOINC'].copy()

count = categorical[['PT_KEY','FEATURE','VALUE']].groupby(['PT_KEY','FEATURE']).sum().reset_index()
count = count.pivot_table(values='VALUE', index='PT_KEY', columns='FEATURE').reset_index()

In [ ]:
agg_val = val_long_data[['PT_KEY']].drop_duplicates().merge(count, how='left', on='PT_KEY')
agg_val = agg_val.merge(median, how='left', on='PT_KEY')
agg_val = agg_val.merge(mean, how='left', on='PT_KEY')
agg_val = agg_val.merge(minimum, how='left', on='PT_KEY')
agg_val = agg_val.merge(maximum, how='left', on='PT_KEY')

agg_val = agg_val.fillna(train_medians.to_dict())
agg_val = agg_val.fillna(0)

agg_val = agg_val.merge(outcomes, how='inner', on='PT_KEY')

### Keep selected columns

In [ ]:
X_val=agg_val[keep_columns].copy()
y_val=agg_val['SSI']

### Scale data

In [ ]:
X_val_scaled = pd.DataFrame(scaler.transform(X_val),columns = X_val.columns)

### Save data

In [ ]:
# X_val_scaled.to_pickle('../../data/val_X_agg.pkl')
# y_val.to_pickle('../../data/val_y_agg.pkl')

# Test Set
### Aggregate data for each patient and feature
Categorical features (like meds, micro results, encounters, dx and px codes) do a count; impute with 0

Numerical features (labs and vitals, terminology = LOINC) get mean, median, min, max; impute with median

In [ ]:
numerical = test_long_data.loc[test_long_data['TERMINOLOGY'] == 'LOINC'].copy()

median = numerical[['PT_KEY','FEATURE','VALUE']].groupby(['PT_KEY','FEATURE']).median().reset_index()
median['FEATURE'] = median['FEATURE'] + '_MEDIAN'
median = median.pivot_table(values='VALUE', index='PT_KEY', columns='FEATURE').reset_index()

mean = numerical[['PT_KEY','FEATURE','VALUE']].groupby(['PT_KEY','FEATURE']).mean().reset_index()
mean['FEATURE'] = mean['FEATURE'] + '_MEAN'
mean = mean.pivot_table(values='VALUE', index='PT_KEY', columns='FEATURE').reset_index()

minimum = numerical[['PT_KEY','FEATURE','VALUE']].groupby(['PT_KEY','FEATURE']).min().reset_index()
minimum['FEATURE'] = minimum['FEATURE'] + '_MIN'
minimum = minimum.pivot_table(values='VALUE', index='PT_KEY', columns='FEATURE').reset_index()

maximum = numerical[['PT_KEY','FEATURE','VALUE']].groupby(['PT_KEY','FEATURE']).max().reset_index()
maximum['FEATURE'] = maximum['FEATURE'] + '_MAX'
maximum = maximum.pivot_table(values='VALUE', index='PT_KEY', columns='FEATURE').reset_index()

In [ ]:
categorical = test_long_data.loc[test_long_data['TERMINOLOGY'] != 'LOINC'].copy()

count = categorical[['PT_KEY','FEATURE','VALUE']].groupby(['PT_KEY','FEATURE']).sum().reset_index()
count = count.pivot_table(values='VALUE', index='PT_KEY', columns='FEATURE').reset_index()

In [ ]:
agg_test = test_long_data[['PT_KEY']].drop_duplicates().merge(count, how='left', on='PT_KEY')
agg_test = agg_test.merge(median, how='left', on='PT_KEY')
agg_test = agg_test.merge(mean, how='left', on='PT_KEY')
agg_test = agg_test.merge(minimum, how='left', on='PT_KEY')
agg_test = agg_test.merge(maximum, how='left', on='PT_KEY')

agg_test = agg_test.fillna(train_medians.to_dict())
agg_test = agg_test.fillna(0)

agg_test = agg_test.merge(outcomes, how='inner', on='PT_KEY')

### Keep selected columns

In [ ]:
X_test=agg_test[keep_columns].copy()
y_test=agg_test['SSI']

### Scale data

In [ ]:
X_test_scaled = pd.DataFrame(scaler.transform(X_test),columns = X_test.columns)

### Save data

In [ ]:
# X_test_scaled.to_pickle('../../data/test_X_agg.pkl')
# y_test.to_pickle('../../data/test_y_agg.pkl')